# Step 2: plot the podeplots

Import all necesary packages

In [ ]:
import numpy as np
import sys, os
import time
import matplotlib.pyplot as plt
import sqlite3 as sql
import scipy as sp

%matplotlib qt

os.chdir(r'd:\Documenten\Documents\GitHub\Active-Vibration-Stabilization\Meetfiles\data\2023-04-23')
print(os.getcwd())

Create the expected fit function

In [ ]:
fit = lambda f, a , b, c, d : f**4 * a+ f**2*b+f**1*c+d

In [ ]:
transferFunction = np.load(f'Full Transfer Function.npy')                                               # Load the transferfunction and remove the outliers 
# transferFunction = np.delete(transferFunction, [12,19],0)                                             # Use if using measurements of 2023-04-17
# transferFunction = np.delete(transferFunction, [2,5,12,14,15,17,24,25,29,41,42,43,44, 46,47,49],0)    # Use if using measurements of 2023-04-18
# transferFunction = np.delete(transferFunction, [11,15,18,22,23,25,29,30,39,49,50],0)                  # Use if using measurements of 2023-04-19
transferFunction = np.delete(transferFunction, [8,14,15,17,18,24,25,27,28,35,40,41,43,44,49],0)       # Use if using measurements of 2023-04-23

frequencies = np.abs(transferFunction[:,:,0])                                                           # get the frequencies
amplitudes = np.abs(transferFunction[:,:,3])                                                            # get the magnitudes 

fitFrequencies = np.linspace(np.min(frequencies),np.max(frequencies), len(frequencies[0])*1000)         # Create an array with much more frequencies for the fit, better caculate the cut-off frequency
fits = [[] for _ in range(len(frequencies[0]))]                                                         # Create an arry for the fits of all DoF

for i in range(len(frequencies[0])):                                                                    # Loop for all DoF
    (fa,fb,fc,fd),_ = sp.optimize.curve_fit(fit, frequencies[:,i], 20*np.log10(amplitudes[:,i]))        # calculate the best fit
    fits[i] = np.append(fits[i],fit(fitFrequencies, fa,fb,fc,fd))                                       # Append the fitted line to the array
    print(fa, fb,fc, fd)                                                                                # print the fit coefficients
fits = np.array(fits)                                                                                   # Transform to a numpy array for better functionallity

fitFrequencies = np.array([fitFrequencies for _ in range(len(frequencies[0]))])                         # Make fitFrequencies compatible with fits

phases = np.rad2deg(np.angle(transferFunction[:,:,3]))                                                  # get the phases and correct them 

# phases[16:,0] -= 360                                          #######################################
# phases[5:,1] -= 360                                           ###                                 ###
# phases[17:,2] -= 360                                          ###                                 ###
# phases[17:,3] -= 360                                          ### Phase correction used for       ###
# phases[[7,8,9,12,14,15,16],3] -= 360                          ###  the measurements of 2023-04-18 ###                 
# phases[[9],4] -= 360                                          ###                                 ###
# phases[14:,4] -= 360                                          ###                                 ###
# phases[4:,5] -= 360                                           #######################################

# phases[29,0] += 360                                           #######################################
# phases[12:,0] -= 360                                          ###                                 ###
# phases[3:,1] -= 360                                           ###                                 ###
# phases[12:,2] -= 360                                          ### Phase correction used for       ###
# phases[5,3] -= 3603                                           ###                                 ###
# phases[7:,3] -= 360                                           ###  the measurements of 2023-04-18 ###
# phases[[8,10,28,30,31,32,33,34],4] -= 360                     ###                                 ###
# phases[12:,4] -= 360                                          ###                                 ###
# phases[1:,5] -= 360                                           #######################################

# phases[[0,2,3],4] += 360                                      #######################################
# phases[11:,[5,1]] -= 360                                      ###                                 ###
# phases[[13,14,15,16,17,18,19,20,27,28,30,32],4] += 360        ### Phase corrections used for      ###
# for i, ph in enumerate(phases):                               ###  the measurements of 2023-04-19 ###
#     for j, p in enumerate(ph):                                ###                                 ###
#         if i > 11:                                            ###                                 ###
#             phases[i,j] =  p -360 if p >0 else p              #######################################

# phases[32:,0] -= 360                                          #######################################
# phases[12:,1] -= 360                                          ###                                 ###
# phases[[13,15],1] += 360                                      ###                                 ###
# phases[12:,2] -= 360                                          ### Phase corrections used for      ###
# phases[[13,15],2] += 360                                      ###                                 ###
# phases[29:,4] -= 360                                          ###  the measurements of 2023-04-23 ###
# phases[[20,21,22],4] -= 360                                   ###                                 ###
# phases[10:,5] -= 360                                          ###                                 ###
# phases[[13,15,17,23],5] += 360                                #######################################

In [ ]:
fig2, (bode_amplitude, bode_phase) = plt.subplots(2,1)
bode_amplitude:plt.Axes
bode_phase:plt.Axes
fig2_manager = plt.get_current_fig_manager()
fig2_manager.set_window_title('bodeplot east Z')
fig2_manager.window.showMaximized()

bode_amplitude.sharex(bode_phase)

bode_amplitude.set_title('East Z actuator Bode plot')
bode_amplitude.set_xscale('log'); bode_amplitude.set_yscale('linear')
bode_amplitude.set_ylabel('Magnitude $(dB)$')
bode_amplitude.set_xlabel('Frequency $(Hz)$')
bode_amplitude.grid(True,'both')
bode_amplitude.set_xlim(0.5*0.95,3.0*1.05)
bode_amplitude.set_ylim(-50*1.05, 10*1.05)

bode_phase.set_ylabel('fase $arg(H(s))$  $[deg]$')
bode_phase.set_xlabel('Frequency $[Hz]$')
bode_phase.grid(True, 'both')
bode_phase.set_ylim(-675,145)

colors = ['#1f77b4','#ff7f0e','#2ca02c','#d62728','#9467bd','#8c564b','#e377c2','#7f7f7f','#bcbd22']
labels = 'x y z rx ry rz'.split()
labelsfit = ['Fitted x','Fitted y','Fitted z','Fitted rx','Fitted ry','Fitted rz']
# labels ='topX topY topZ northX northY northZ eastX eastY eastZ'.split()
# labelsfit = ['Fitted topX','Fitted topY','Fitted topZ','Fitted northX','Fitted northY','Fitted northZ','Fitted eastX','Fitted eastY','Fitted eastZ']

for f,a,p,ff,af,c,l,lf in zip(np.transpose(frequencies), np.transpose(20*np.log10(amplitudes)), np.transpose(phases), fitFrequencies, fits, colors, labels, labelsfit):
    _3db = (np.max(af)-6)*np.ones_like(ff)
    _3dbi = np.where((af - _3db) >= 0)[0]
    
    i = np.where((_3dbi[1:] - _3dbi[:-1]) != 1)[0]
    _3dbi = _3dbi[:i[0]+1] if len(i) > 0 else _3dbi
    f_min = ff[_3dbi[0]]
    f_max = ff[_3dbi[-1]]
    f_res = ff[np.argmax(af)]
    Q = f_res / np.abs(f_max-f_min)

    print(f'''
{l}
    f_res: {f_res:.2f}
    f_min: {f_min:.2f}
    f_max: {f_max:.2f}
    q_fac: {Q:.2f}
    ''')

    # bode_amplitude.plot(ff,af,color = c,linestyle='-.',label=lf)
    bode_amplitude.plot(f, a, color = c, label=l, marker='o', linestyle='-')
    # bode_amplitude.plot(ff[_3dbi], _3db[_3dbi],linestyle=':', color = c)
    bode_phase.plot(f, p, c,label=l, marker='o', linestyle='-')
    # for n, (ph, fp) in enumerate(zip(p,f)):
    #     bode_phase.annotate(f'{n}', (fp,ph))
        
bode_amplitude.legend()
bode_phase.legend()


# bode_amplitude.plot(frequencies, amp, '-', frequencies, _3db, ':')

plt.pause(0.002)
fig2.tight_layout()